# classify gene list bo GO groups
uses file from UniProt, called for GO in ASlist (by Arthur Svendsen)
The script below is by L.V.Bystrykh

first define file read function for simplicity 

In [1]:
def getitems(file, sep):
    f_in=open(file,"r")
    lines=f_in.readlines()
    items=[x.rstrip().split(sep) for x in lines]
    return(items)

Get gene list from the Aging List, cut by frequency >3, which is defined as Aging Signature

In [2]:
AL=getitems("/Users/Art/Drive/PhD/Experiments/Aging Signature/000_FINAL/00_sourceFiles/01_Aging_List_REAN.txt","\t")[1:]
ALgenes=[x[0] for x in AL if float(x[1])>3]
#ALfreqs=[x[1] for x in AL if float(x[1])>3] not used in current script
ALgenes.remove("") #remove empty gene name
print("list ALgenes", len(ALgenes))

list ALgenes 221


Next, open Uniprot GO list, check length

In [3]:
dr="/Users/Art/Drive/PhD/Experiments/Aging Signature/000_FINAL/00_sourceFiles/Uniprot/"
fl="AS_Uniprot_query_GO_filtered.txt"
#compile all GO names
all_GO, all_GO_genes=[],[] #all GO groups redundant
AS_GO=getitems(dr+fl,"\t")
(print("GO gene list", len(AS_GO)))

GO gene list 218


collect all GO items and check if some genes are NOT in the original AS list

In [4]:
for line in AS_GO:
    if line[1] not in ALgenes:
        print(line[1], "not in AS list")
    if line[1] not in all_GO_genes:
        all_GO_genes.append(line[1])
    go_items=line[2].split(";")
    for g in go_items:
        all_GO.append(g)
        

Fyb1 not in AS list
Nt5c3a not in AS list
Phf11 not in AS list
Serpinb6 not in AS list
Tlcd4 not in AS list


Unfortunately some genes are not as we expected. We can do some manual corrections but currently we don't.
Let's also check if any genes from AS list are missing in GO lists

In [5]:
missing=[x for x in ALgenes if x not in all_GO_genes]
print("missing in GO gene names", missing)

missing in GO gene names ['Mt2', 'Phf11d', 'Pcdhb16', 'Tmem56', 'Fyb', 'Serpinb6a', 'Sdpr', 'Nt5c3']


You can recognise similar gene names like *Fyb* and *Fyb1*. This is what usually happens with GO search.
The next step is to count all GO groups, sort them, check the most popular (first 5 as defined now).

In [6]:
"count all GO names"
nr_GO, all_FR=[],[]
for item in set(all_GO):
    nr_GO.append(item)
    freqs=all_GO.count(item)
    all_FR.append(freqs)
sort=sorted(zip(all_FR,nr_GO), reverse=True)
print(len(sort))
for item in sort:
    print(item)

2336
(75, ' plasma membrane [GO:0005886]')
(56, ' nucleus [GO:0005634]')
(44, ' integral component of membrane [GO:0016021]')
(41, ' cytosol [GO:0005829]')
(38, ' cytoplasm [GO:0005737]')
(32, 'cytoplasm [GO:0005737]')
(32, ' extracellular space [GO:0005615]')
(29, ' identical protein binding [GO:0042802]')
(26, ' protein homodimerization activity [GO:0042803]')
(26, ' integral component of plasma membrane [GO:0005887]')
(24, ' membrane [GO:0016020]')
(22, ' nucleoplasm [GO:0005654]')
(21, ' calcium ion binding [GO:0005509]')
(19, ' perinuclear region of cytoplasm [GO:0048471]')
(19, ' metal ion binding [GO:0046872]')
(18, ' cell surface [GO:0009986]')
(17, ' protein-containing complex [GO:0032991]')
(17, ' positive regulation of cell population proliferation [GO:0008284]')
(17, ' ATP binding [GO:0005524]')
(16, ' positive regulation of transcription by RNA polymerase II [GO:0045944]')
(16, ' positive regulation of apoptotic process [GO:0043065]')
(15, 'integral component of membrane [

We can immediately see the gene numbers for each GO group, although we do not know what genes are there.
First, we can study those GO groups in details and define the most interesting (for our context of the study). We add those key terms in the list of "keys" below.
Note that we use *find* function, which is exact match. So redefine the search or key if you want something more sofisticated.
The loop below will printout many things, including lists of found genes and not found genes, GO groups for all found genes

In [18]:
#check GO categories
selected, genes=[],[] #selected GO groups, genes found in GO groups
keys=["membran", "cell cycl", "histon","inflammat"]
for key in keys:
    sel, gen=[],[]
    #find all GO groups with given key
    for item in sort:
        if item[1].find(key) >= 0:
            sel.append(item[1])
    print("search item: ", key)
    selected.append(sel) #group of GOs per key
    #print("selected", str(sel))
    print("number of GOs:", len(sel))
    print("number of GOs:", len(sel))
    #find genes for each GO category
    sel_lines=[]
    for line in AS_GO:
        for item in sel:
            if line[2].find(item)>=0:
                if line not in sel_lines:
                    sel_lines.append(line)
                if line[1] not in genes:
                    genes.append(line[1])
    not_found_genes=[x for x in ALgenes if x not in genes]    
#     print("cumulatively found genes", len(genes), "not found", len(not_found_genes))
    #print("found", genes)
    #print("not found", not_found_genes)
#     for i in range(len(sel_lines)):
#         print(i, str(sel_lines[i])[:80])

search item:  membran
number of GOs: 138
search item:  cell cycl
number of GOs: 15
search item:  histon
number of GOs: 7
search item:  inflammat
number of GOs: 8


The printout is a bit messy and ugly. I truncated the lines to make it a bit more readable. If needed, I can add lines to save in file.